# CWRU Bearing Fault Data - Data Feature Extraction (divided by load)

In [198]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import scipy.io
import pickle
import librosa

from scipy.stats import skew, kurtosis, entropy
from scipy.fft import fft
from scipy.signal import stft
from IPython.display import display_html

In [200]:
def extract_time_domain_features(signal):
    signal = np.array(signal)

    zcr = librosa.feature.zero_crossing_rate(signal, frame_length=len(signal), hop_length=len(signal))[0][0]
    rms = np.sqrt(np.mean(signal**2))
    mean = np.mean(signal)
    avg = np.mean(np.abs(signal))

    return {
        'mean': mean,
        'std': np.std(signal),
        'variance': np.var(signal),
        'min': np.min(signal),
        'max': np.max(signal),
        'peak_to_peak': np.ptp(signal),
        'skewness': skew(signal),
        'kurtosis': kurtosis(signal),
        'rms': np.sqrt(np.mean(signal**2)), # root mean square
        'zcr': zcr, # zero crossing rate
        'crest_factor': np.max(np.abs(signal)) / rms,
        'form_factor': rms / avg
    }

In [201]:
def extract_frequency_domain_features(fft_amplitudes):
    fft_amplitudes = np.array(fft_amplitudes)
    freqs = np.linspace(0, SAMPLE_RATE / 2, len(fft_amplitudes))

    norm_fft_amplitudes = fft_amplitudes / (np.sum(fft_amplitudes) + 1e-12)
    power_spectrum = fft_amplitudes**2

    peak_idx = np.argmax(fft_amplitudes)
    peak_freq = freqs[peak_idx]

    cumulative_power = np.cumsum(power_spectrum)
    median_freq = freqs[np.searchsorted(cumulative_power, cumulative_power[-1] / 2)]

    low_band = power_spectrum[freqs < 3000]
    high_band = power_spectrum[freqs >= 3000]
    band_energy_ratio = np.sum(high_band) / (np.sum(low_band) + 1e-12)

    return {
        'mean': np.mean(fft_amplitudes), # mean magnitude
        'std': np.std(fft_amplitudes), # standard deviation magnitude
        'max': np.max(fft_amplitudes), # max magnitude
        'skewness': skew(fft_amplitudes), # skewness magnitude
        'kurtosis': kurtosis(fft_amplitudes), # kurtosis magnitude
        'spec_centroid': np.sum(freqs * norm_fft_amplitudes), # spectral centroid
        'spec_spread': np.sqrt(np.sum(((freqs - np.sum(freqs * norm_fft_amplitudes))**2) * norm_fft_amplitudes)), # spectral_spread
        'spec_flatness': np.exp(np.mean(np.log(fft_amplitudes + 1e-12))) / (np.mean(fft_amplitudes) + 1e-12), # spectral_flatness
        'spec_entropy': entropy(norm_fft_amplitudes), # spectral_entropy
        'total_power': np.sum(power_spectrum),
        'dominant_freq': peak_freq, # dominant frequency
        'median_freq': median_freq, # median frequency
        'ber': band_energy_ratio, # band energy ratio
    }

In [202]:
def extract_time_frequency_domain_features(magnitude, freqs):
    mean_mag = np.mean(magnitude)
    max_mag = np.max(magnitude)
    spec_centroid = np.sum(freqs[:, None] * magnitude) / (np.sum(magnitude) + 1e-12)
    spec_spread = np.sqrt(np.sum(((freqs[:, None] - spec_centroid)**2) * magnitude) / (np.sum(magnitude) + 1e-12))
    total_power = np.sum(magnitude**2)

    return {
        'mean': mean_mag, # mean magnitude
        'max': max_mag, # max_magnitude
        'spec_centroid': spec_centroid, # spectral centroid
        'spec_spread': spec_spread, # spectral spread
        'total_power': total_power
    }

## 48k Drive-End Bearing Fault Data + Normal Baseline Data (48k)

In [ ]:
with open('48kdrive-end_normalbaseline_loadsplit_cropped_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [204]:
SAMPLE_RATE = 48000
FRAME_SIZE = 1024
HOP_SIZE = 512

### Feature extraction in time domain

In [205]:
load_feature_tables = {}

for load in data['load'].unique():
    load_df = data[data['load'] == load]

    feature_rows = []
    for _, row in load_df.iterrows():
        features = extract_time_domain_features(row['cropped_signal'])
        features['label'] = row['label']
        feature_rows.append(features)

    features_df = pd.DataFrame(feature_rows)
    load_feature_tables[load] = features_df

#### Mean, Standard deviation, Variance

In [206]:
tables = []

for load in sorted(load_feature_tables.keys()):
    df = load_feature_tables[load][['label', 'mean', 'variance', 'std']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,mean,variance,std
Normal,0.012255,0.005173,0.071921
B007,0.003296,0.007124,0.084405
B014,0.002709,0.021903,0.147998
B021,0.002516,0.010058,0.100289
OR007@12,0.003410,0.009902,0.099508
OR007@3,0.002382,0.007430,0.086197
OR014@3,0.003015,0.039618,0.199043
IR007,0.003115,0.014987,0.122420
IR014,0.003726,0.033691,0.183550
IR021,0.012099,0.040213,0.200531


#### Min, Max, Peak to Peak

In [207]:
tables = []

for load in sorted(load_feature_tables.keys()):
    df = load_feature_tables[load][['label', 'min', 'max', 'peak_to_peak']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:8px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,min,max,peak_to_peak
Normal,-0.286638,0.311254,0.597892
B007,-0.693111,0.569335,1.262446
B014,-1.504555,1.549225,3.053780
B021,-0.596949,0.776034,1.372983
OR007@12,-0.480771,0.482070,0.962841
OR007@3,-0.427627,0.403065,0.830692
OR014@3,-1.186426,1.173594,2.360020
IR007,-0.653801,0.717314,1.371115
IR014,-1.138508,0.972662,2.111169
IR021,-1.135702,1.016791,2.152494


#### Skewness, Kurtosis

In [208]:
tables = []

for load in sorted(load_feature_tables.keys()):
    df = load_feature_tables[load][['label', 'skewness', 'kurtosis']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,skewness,kurtosis
Normal,-0.036569,-0.219030
B007,0.022704,0.869712
B014,0.027552,8.303194
B021,-0.024162,0.473043
OR007@12,0.004946,0.351325
OR007@3,0.007862,0.240899
OR014@3,-0.002620,3.294376
IR007,-0.151455,1.011153
IR014,0.004744,0.709505
IR021,-0.033539,1.645847


#### RMS (root mean square)

In [209]:
tables = []

for load in sorted(load_feature_tables.keys()):
    df = load_feature_tables[load][['label', 'rms']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:100px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,rms
Normal,0.072957
B007,0.084469
B014,0.148023
B021,0.100321
OR007@12,0.099567
OR007@3,0.086230
OR014@3,0.199066
IR007,0.122460
IR014,0.183588
IR021,0.200896


#### Zero crossing rate

In [210]:
tables = []

for load in sorted(load_feature_tables.keys()):
    df = load_feature_tables[load][['label', 'zcr']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:100px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,zcr
Normal,0.083322
B007,0.224570
B014,0.222207
B021,0.202459
OR007@12,0.191797
OR007@3,0.222862
OR014@3,0.246292
IR007,0.212284
IR014,0.235514
IR021,0.249476


### Feature extraction in frequency domain

In [211]:
load_fft_tables = {}
load_fft_feature_tables = {}

for load in data['load'].unique():
    load_df = data[data['load'] == load]

    fft_rows = []
    feature_rows = []

    for _, row in load_df.iterrows():
        signal = np.array(row['cropped_signal'])

        fft_result = np.fft.rfft(signal)
        fft_amplitudes = np.abs(fft_result)

        fft_dict = {f'fft_{i}': amp for i, amp in enumerate(fft_amplitudes)}
        fft_dict['label'] = row['label']
        fft_rows.append(fft_dict)

        features = extract_frequency_domain_features(fft_amplitudes)
        features['label'] = row['label']
        feature_rows.append(features)

    fft_df = pd.DataFrame(fft_rows)
    features_df = pd.DataFrame(feature_rows)

    load_fft_tables[load] = fft_df
    load_fft_feature_tables[load] = features_df

#### Mean magnitude, Standard deviation magnitude, Max magnitude

In [212]:
tables = []

for load in sorted(load_fft_feature_tables.keys()):
    df = load_fft_feature_tables[load][['label', 'mean', 'std', 'max']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:2px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,mean,std,max
Normal,5.587756,25.077865,2847.158654
B007,14.000226,25.805048,1430.498791
B014,25.820830,44.462893,1491.843175
B021,14.485033,31.699450,1246.410013
OR007@12,12.993907,32.066976,1355.260562
OR007@3,12.597450,27.181685,879.101811
OR014@3,22.559158,65.359334,3664.400126
IR007,17.276563,38.877885,1507.910033
IR014,22.807117,59.555260,4636.998313
IR021,25.818883,64.953944,2878.160274


#### Skewness magnitude, Kurtosis magnitude

In [213]:
tables = []

for load in sorted(load_fft_feature_tables.keys()):
    df = load_fft_feature_tables[load][['label', 'skewness', 'kurtosis']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,skewness,kurtosis
Normal,55.847720,4948.632543
B007,9.533126,261.008620
B014,5.283822,63.788199
B021,9.550278,179.434736
OR007@12,12.508942,288.464694
OR007@3,8.715504,135.228527
OR014@3,15.957681,462.731979
IR007,12.755724,292.278754
IR014,26.167320,1403.010742
IR021,8.899599,148.637371


#### Spectral centroid, Spectral spread

In [214]:
tables = []

for load in sorted(load_fft_feature_tables.keys()):
    df = load_fft_feature_tables[load][['label', 'spec_centroid', 'spec_spread']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:8px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,spec_centroid,spec_spread
Normal,4786.205791,4558.571856
B007,11318.681093,5043.808002
B014,11390.116546,4490.772684
B021,10559.446021,4486.844478
OR007@12,10502.146436,4788.070183
OR007@3,10935.424380,4728.236287
OR014@3,12058.877638,4723.507206
IR007,10744.044357,5013.792280
IR014,11117.292852,4781.939400
IR021,11713.940921,4674.767447


#### Spectral flatness, Spectral entropy

In [215]:
tables = []

for load in sorted(load_fft_feature_tables.keys()):
    df = load_fft_feature_tables[load][['label', 'spec_flatness', 'spec_entropy']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:8px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,spec_flatness,spec_entropy
Normal,0.266110,9.691646
B007,0.462029,10.225548
B014,0.437961,10.215167
B021,0.384191,10.043670
OR007@12,0.397802,10.012729
OR007@3,0.411708,10.084155
OR014@3,0.399520,9.922678
IR007,0.438431,10.133464
IR014,0.408803,10.065798
IR021,0.385548,9.925622


#### Total power, Median frequency

In [216]:
tables = []

for load in sorted(load_fft_feature_tables.keys()):
    df = load_fft_feature_tables[load][['label', 'total_power', 'median_freq']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:8px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,total_power,median_freq
Normal,3.981132e+07,4144.325794
B007,5.198074e+07,12897.791338
B014,1.594367e+08,11979.704185
B021,7.325561e+07,11012.668303
OR007@12,7.219787e+07,10679.976123
OR007@3,5.412972e+07,11920.408569
OR014@3,2.883227e+08,12941.964582
IR007,1.091575e+08,11186.177622
IR014,2.452763e+08,12866.750680
IR021,2.946474e+08,13091.596471


#### Dominant frequency, Band energy ratio

In [217]:
tables = []

for load in sorted(load_fft_feature_tables.keys()):
    df = load_fft_feature_tables[load][['label', 'dominant_freq', 'ber']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)

label,dominant_freq,ber
Normal,4144.325794,2.081370
B007,5750.082908,4.824897
B014,5746.501293,13.430146
B021,1439.411023,5.921656
OR007@12,2035.550839,4.477783
OR007@3,4309.478013,7.965912
OR014@3,12976.984811,22.920480
IR007,16491.344432,12.128873
IR014,12866.750680,7.743924
IR021,13737.878888,14.245164


### Feature extraction in time-frequency domain

In [218]:
load_stft_tables = {}
load_stft_feature_tables = {}

for load in sorted(data['load'].unique()):
    load_df = data[data['load'] == load]

    stft_rows = []
    feature_rows = []

    for _, row in load_df.iterrows():
        signal = np.array(row['cropped_signal'])

        S = librosa.stft(signal, n_fft=FRAME_SIZE, hop_length=HOP_SIZE, win_length=FRAME_SIZE, window='hann', center=True)
        magnitude = np.abs(S)

        freqs = np.linspace(0, SAMPLE_RATE / 2, magnitude.shape[0])

        stft_dict = {f'stft_{i}': mag for i, mag in enumerate(magnitude.flatten())}
        stft_dict['label'] = row['label']
        stft_rows.append(stft_dict)

        features = extract_time_frequency_domain_features(magnitude, freqs)
        features['label'] = row['label']
        feature_rows.append(features)

    stft_df = pd.DataFrame(stft_rows)
    features_df = pd.DataFrame(feature_rows)

    load_stft_tables[load] = stft_df
    load_stft_feature_tables[load] = features_df


#### Mean magnitude for all frames, Max magnitude for all frames

In [219]:
tables = []

for load in sorted(load_stft_feature_tables.keys()):
    df = load_stft_feature_tables[load][['label', 'mean', 'max']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)


label,mean,max
Normal,0.533502,16.965659
B007,0.998792,19.476653
B014,1.552758,44.715976
B021,1.132916,19.985390
OR007@12,1.181489,17.356743
OR007@3,1.018160,16.269384
OR014@3,1.950357,43.516096
IR007,1.561926,19.096540
IR014,1.917840,42.580971
IR021,1.963516,48.446957


#### Spectral centroid for all frames, Spectral spread for all frames

In [220]:
tables = []

for load in sorted(load_stft_feature_tables.keys()):
    df = load_stft_feature_tables[load][['label', 'spec_centroid', 'spec_spread']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)


label,spec_centroid,spec_spread
Normal,4219.490662,4021.127323
B007,10079.166272,5613.627420
B014,10601.770771,4986.854959
B021,9315.429516,4821.564089
OR007@12,9236.851512,5115.439148
OR007@3,10017.620286,5136.682492
OR014@3,11305.314596,4942.818666
IR007,9894.611208,5212.324311
IR014,10241.007015,5207.394045
IR021,10767.417547,5108.149848


#### Total power for all frames

In [221]:
tables = []

for load in sorted(load_stft_feature_tables.keys()):
    df = load_stft_feature_tables[load][['label', 'total_power']].copy()
    html_table = df.to_html(index=False)
    tables.append(f'<div style="display:inline-block; margin-right:30px;">{html_table}</div>')

display_html(''.join(tables), raw=True)


label,total_power
Normal,2.508656e+05
B007,3.312226e+05
B014,1.016189e+06
B021,4.671447e+05
OR007@12,4.589062e+05
OR007@3,3.455239e+05
OR014@3,1.828605e+06
IR007,6.942076e+05
IR014,1.563656e+06
IR021,1.876308e+06
